In [1]:
# mount drive for access to the files
from google.colab import drive

drive.mount("/content/drive")

# all the drive the files are present in "/content/drive/My Drive"
!ls "/content/drive/My Drive/Beuth Uni/Master Thesis"

import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Data
'Data Quality in ML Production Systems.pdf'
'Datawig: Missing Value Imputation for Tables.pdf'
 Declaration
 Images
 jenga
 jenga.pdf
 MICE_Multivariate_Imputation_by_Chained_Equations_.pdf


In [2]:
!pip install openml
!pip install pyod

!pip install mxnet autogluon
!pip install mxnet-mkl --pre --upgrade

Requirement already up-to-date: mxnet-mkl in /usr/local/lib/python3.6/dist-packages (1.6.0)


In [3]:
import random
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNNOutlierDetection, PyODIsolationForestOutlierDetection, AutoGluonOutlierDetection
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, AutoGluonImputation
from jenga.cleaning.clean import Clean

In [4]:
seed = 10

In [5]:
def run_experiment(dataset_name, learner, param_grid, corruptions, fraction, cleaners, num_repetitions, categorical_precision_threshold=0.7, numerical_std_error_threshold=2.0):
    
    ## dataset
    dataset = Dataset(seed, dataset_name)
    
    all_data = dataset.all_data
    attribute_names = dataset.attribute_names
    attribute_types = dataset.attribute_types
    
    ## categorical and numerical features
    categorical_columns = dataset.categorical_columns
    numerical_columns = dataset.numerical_columns
    print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")
    
    ## train and test data
    df_train, lab_train, df_test, lab_test = dataset.get_train_test_data()
    
    
    ## pipeline performance prediction (ppp)
    ppp = PipelinePerformancePrediction(seed, df_train, lab_train, df_test, lab_test, categorical_columns, numerical_columns, learner, param_grid)
    ppp_model = ppp.fit_ppp(df_train)
    
    ## generate corrpted data
    df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(df_test, corruptions, fraction, num_repetitions)
    
    ## cleaning
    clean = Clean(df_train, df_corrupted, categorical_columns, numerical_columns, categorical_precision_threshold, numerical_std_error_threshold, ppp, ppp_model, cleaners)
    df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(df_train, df_corrupted)
    
    ## results
    result = {
        'ppp_score_model': ppp.predict_score_ppp(ppp_model, df_test),
        'ppp_score_corrupted': corrupted_score_ppp,
        'ppp_score_cleaned': best_cleaning_score,
        'ppp_scores_cleaners': cleaner_scores_ppp
    }
#     print('\n'.join([f'{key}:{val}' for key, val in result.items()]))
    
    ## summary
    summary = {
        'dataset': dataset_name,
        'model': learner,
        'corruptions': summary_col_corrupt,
        'cleaners': summary_cleaners,
        'result': result
    }
#     print('\n\n\n\n'.join([f'{key}:{val}' for key, val in summary.items()]))
    
    return summary #summary_col_corrupt, result

In [6]:
datasets = [
    'parkinsons',
    'heart-statlog',
    # 'credit-g'
]

In [7]:
## model parameters
## models is a dict where key = leaner & value = param_grid
models = {SGDClassifier(loss='log'): {'learner__max_iter': [500, 1000, 5000], 
                                         'learner__penalty': ['l2', 'l1', 'elasticnet'], 
                                         'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
                                        }, 
          RandomForestClassifier():{'learner__n_estimators': [100, 200, 500], 
                                    'learner__max_depth': [5, 10, 15]
                                   }
         }

## make dict of multiple leraners and corresponding param_grids

In [8]:
corruptions = [MissingValues, SwappedValues, Scaling, GaussianNoise]

In [9]:
fractions = np.random.uniform(0, 1, 2) # np.random.uniform(0, 1, 3)

In [10]:
cleaners = [
    (NoOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, AutoGluonImputation),
    (PyODIsolationForestOutlierDetection, MeanModeImputation),
    (PyODIsolationForestOutlierDetection, AutoGluonImputation),
    (AutoGluonOutlierDetection, MeanModeImputation),
    (AutoGluonOutlierDetection, AutoGluonImputation)
]

In [11]:
%%time
for _ in range(1):
  print("\n\n..................................ITERATION..................................\n")
  ind_results = []

  for dataset in datasets:
      for learner, param_grid in models.items():
          for fraction in fractions:
              # ind_results.append(run_experiment(dataset, learner, param_grid, [MissingValues], fraction, [(PyODKNN, AutoGluonImputation)], 5))
              ind_results.append(run_experiment(dataset, learner, param_grid, [MissingValues], fraction, cleaners, 100))



..................................ITERATION..................................



Saved dataset 1488: parkinsons to file /root/.openml/cache/org/openml/www/datasets/1488/dataset.pkl.py3


Dataset: parkinsons
Found 0 categorical and 22 numeric features 

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    3.4s finished



Generating corrupted training data on 39 rows... 

	perturbation: MissingValues: {'column': 'V2', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V16', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V1', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V15', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V9', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MAR'}
	perturbation: MissingValues: {'column': 'V6', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MNAR'}
	perturbation: MissingValues: {'column': 'V17', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingness': 'MCAR'}
	perturbation: MissingValues: {'column': 'V11', 'fraction': 0.15302909952716925, 'na_value': nan, 'missingn

...

Cleaner: {'outlier_detection': PyODIsolationForestOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.9203296703296704, 'classification_report': {'absent': {'precision': 0.7741935483870968, 'recall': 0.9230769230769231, 'f1-score': 0.8421052631578947, 'support': 26}, 'present': {'precision': 0.9130434782608695, 'recall': 0.75, 'f1-score': 0.8235294117647057, 'support': 28}, 'accuracy': 0.8333333333333334, 'macro avg': {'precision': 0.8436185133239831, 'recall': 0.8365384615384616, 'f1-score': 0.8328173374613002, 'support': 54}, 'weighted avg': {'precision': 0.8461898083216456, 'recall': 0.8333333333333334, 'f1-score': 0.8324733402132781, 'support': 54}}}
Column age: Num NaNs: Before: 0, Now: 0
Column sex: Num NaNs: Before: 0, Now: 0
Column chest: Num NaNs: Before: 0, Now: 5
Column resting_blood_pressure: Num NaNs: Before: 0, Now: 8
Column serum_cholestoral: Num NaNs: Before: 0, Now: 0
Column fasting_blood_sugar: Num NaNs: Before: 0, Now: 11
Column resting_electroc

/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 2 (out of 52) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation datapoints for which the label is missing. "
Evaluation: root_mean_squared_error on test data: 6.741113838908102
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 7 (out of 47) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation

Cleaner: {'outlier_detection': AutoGluonOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.9230769230769231, 'classification_report': {'absent': {'precision': 0.7741935483870968, 'recall': 0.9230769230769231, 'f1-score': 0.8421052631578947, 'support': 26}, 'present': {'precision': 0.9130434782608695, 'recall': 0.75, 'f1-score': 0.8235294117647057, 'support': 28}, 'accuracy': 0.8333333333333334, 'macro avg': {'precision': 0.8436185133239831, 'recall': 0.8365384615384616, 'f1-score': 0.8328173374613002, 'support': 54}, 'weighted avg': {'precision': 0.8461898083216456, 'recall': 0.8333333333333334, 'f1-score': 0.8324733402132781, 'support': 54}}}
Column age: Num NaNs: Before: 0, Now: 2
Column sex: Num NaNs: Before: 0, Now: 7
Column chest: Num NaNs: Before: 0, Now: 0
Column resting_blood_pressure: Num NaNs: Before: 0, Now: 1
Column serum_cholestoral: Num NaNs: Before: 0, Now: 1
Column fasting_blood_sugar: Num NaNs: Before: 0, Now: 11
Column resting_electrocardiograph

/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 2 (out of 52) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation datapoints for which the label is missing. "
Evaluation: root_mean_squared_error on test data: 6.741113838908102
/usr/local/lib/python3.6/dist-packages/autogluon/utils/tabular/ml/learner/abstract_learner.py:360: UserWarning: There are 7 (out of 47) evaluation datapoints for which the label is missing. AutoGluon removed these points from the evaluation, which thus may not be entirely representative. You should carefully study why there are missing labels in your evaluation data.
  warnings.warn(f"There are {n_missing} (out of {len(y_true)}) evaluation


Best cleaning method:
Cleaning score: Cleaner: {'outlier_detection': AutoGluonOutlierDetection, 'imputation': AutoGluonImputation}: {'roc_auc_acore': 0.9230769230769231, 'classification_report': {'absent': {'precision': 0.7741935483870968, 'recall': 0.9230769230769231, 'f1-score': 0.8421052631578947, 'support': 26}, 'present': {'precision': 0.9130434782608695, 'recall': 0.75, 'f1-score': 0.8235294117647057, 'support': 28}, 'accuracy': 0.8333333333333334, 'macro avg': {'precision': 0.8436185133239831, 'recall': 0.8365384615384616, 'f1-score': 0.8328173374613002, 'support': 54}, 'weighted avg': {'precision': 0.8461898083216456, 'recall': 0.8333333333333334, 'f1-score': 0.8324733402132781, 'support': 54}}} 

Cleaning didnt't improve the overall score 



CPU times: user 1h 18min 46s, sys: 4min 18s, total: 1h 23min 5s
Wall time: 1h 10min 23s


In [12]:
ind_results

[{'cleaners': [{'Imputation method': MeanModeImputation,
    'Outlier detection method': NoOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'1': {'f1-score': 0.9090909090909091,
       'precision': 0.9090909090909091,
       'recall': 0.9090909090909091,
       'support': 11},
      '2': {'f1-score': 0.9642857142857143,
       'precision': 0.9642857142857143,
       'recall': 0.9642857142857143,
       'support': 28},
      'accuracy': 0.9487179487179487,
      'macro avg': {'f1-score': 0.9366883116883117,
       'precision': 0.9366883116883117,
       'recall': 0.9366883116883117,
       'support': 39},
      'weighted avg': {'f1-score': 0.9487179487179487,
       'precision': 0.9487179487179487,
       'recall': 0.9487179487179487,
       'support': 39}},
     'roc_auc_acore': 0.9902597402597403}},
   {'Imputation method': MeanModeImputation,
    'Outlier detection method': PyODKNNOutlierDetection,
    'PPP score with cleaning': {'classification_report': {'

In [13]:
# %%time
# for _ in range(1):
#   print("\n\n..................................ITERATION..................................\n")
#   results = []

#   for dataset in datasets:
#       for learner, param_grid in models.items():
#           for fraction in fractions:
#               results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 5))

In [14]:
# results